In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import glob
from tqdm import tqdm
from pandas.tseries.offsets import DateOffset
from bisect import bisect
import gc
from datetime import datetime
import cv2

In [ ]:
from netCDF4 import Dataset
from netCDF4 import num2date

In [ ]:
import sys
sys.path.append("./src") 

In [ ]:
from src.firms_tools import (
    process_firms_chunk,
    aggregate_month,
    add_lat_lon_idx_to_fires,
    get_tiles_df,
    get_lats_lons,
)
from src.config import IMAGE_SIZE, SOUTH_AMERICA, south_america_coordinates, FIRECCI_SHAPE

# FIRMS

In [ ]:
t0 = datetime.now()
FIRMS_DIR = '/home/gabor/h2o/corner-beluga/projects/fire/firms_data'
firms_files = glob.glob(f'{FIRMS_DIR}/**/*.csv', recursive=True)
len(firms_files)

In [ ]:
if not os.path.exists('firms_filestats.csv'):
    rows = []
    for f in tqdm(firms_files):
        cols_to_read = [
            "latitude",
            "longitude",
            "acq_date",
            "satellite",
            "instrument",
            "version",
            "confidence",
        ]
        df = pd.read_csv(f, usecols=cols_to_read, parse_dates=['acq_date'], low_memory=False)
        csv_name = f.split('/')[-1]
        row = [
            f, csv_name, df.shape[0], df.shape[1], df.acq_date.min(), df.acq_date.max(),
            df.satellite.max(), df.instrument.max(), df.version.max(),
            df.latitude.nunique(), df.longitude.nunique(),
            df.confidence.nunique(), df.satellite.nunique(), df.acq_date.nunique()
        ]
        rows.append(row)

    cols = [
        'path', 'csv', 'rows', 'cols', 'start', 'end',
        'satellite', 'instrument', 'version',
        'lats', 'lons', 'confs', 'sats', 'days'
    ]
    filestats = pd.DataFrame(rows, columns=cols)
    filestats = filestats.sort_values(by=['start', 'instrument'])
    filestats
    filestats.to_csv('firms_filestats.csv', index=False)

In [ ]:
filestats = pd.read_csv('firms_filestats.csv')
filestats
filestats['year'] = pd.to_datetime(filestats.start).dt.year
px.bar(
    filestats, x='year', y='rows', hover_name='csv', color='instrument',
    title='FIRMS records'
)

# Process each chunk

We removed fire readings with low or less than 50 confidence. For simplicity the coordinates are rounded to three decimal degrees. That is roughly 110 m at the Equator. For better spatial resolution the original VIIRS records could be used.

In [ ]:
south_america_coordinates

In [ ]:
lat_min = min([lat for lon, lat in south_america_coordinates[0]])
lat_max = max([lat for lon, lat in south_america_coordinates[0]])
lon_min = min([lon for lon, lat in south_america_coordinates[0]])
lon_max = max([lon for lon, lat in south_america_coordinates[0]])
lat_min, lat_max
lon_min, lon_max

In [ ]:
if not os.path.exists('sa_fires.csv'):
    chunks = []
    for f in tqdm(firms_files):
        daily_fires = process_firms_chunk(f)
        # filter years
        df = daily_fires[daily_fires.acq_date.dt.year >= 2013].copy()
        df = df[df.acq_date.dt.year < 2021]
        
        # filter region
        df = df[df.latitude >= lat_min]
        df = df[df.latitude <= lat_max]
        
        df = df[df.longitude >= lon_min]
        df = df[df.longitude <= lon_max]
        if len(df) > 0:
            print(daily_fires.acq_date.min(), daily_fires.shape, df.shape)
            chunks.append(df)

    full_dataset = pd.concat(chunks)
    full_dataset.shape
    full_dataset.head()

    del chunks
    gc.collect()

    fires = aggregate_month(full_dataset)
    fires.shape
    fires.head()
    fires.to_csv('sa_fires.csv', index=False)
    
    del full_dataset
    gc.collect()
    
else:
    fires = pd.read_csv('sa_fires.csv')

In [ ]:
df

In [ ]:
fires.info()

In [ ]:
fires.head()
fires.shape

In [ ]:
yearly_fires = fires[fires.year < 2022].groupby(['year', 'month']).sum().reset_index()
yearly_fires.head()
px.bar(yearly_fires, x='month', y='fire_cnt', color='year',
       title='Hotspot detections Worldwide')

# Satellite Images

In [ ]:
REGION = SOUTH_AMERICA

In [ ]:
lats, lons = get_lats_lons(REGION)
lats.min(), lats.max()
lons.min(), lons.max()
lats.shape, lons.shape

In [ ]:
tiles_df = get_tiles_df(REGION)

tiles_df.shape
tiles_df.head()

In [ ]:
fires_on_map = fires[
    (fires.latitude >= tiles_df.lat_min.min()) & \
    (fires.latitude <= tiles_df.lat_max.max()) & \
    (fires.longitude >= tiles_df.lon_min.min()) & \
    (fires.longitude <= tiles_df.lon_max.max()) 

].copy()
fires_on_map.shape
fires_on_map.head()

In [ ]:
del fires
gc.collect()

In [ ]:
fires_on_map = add_lat_lon_idx_to_fires(fires_on_map, tiles_df, lats, lons)

In [ ]:
np.mean(fires_on_map.lat_idx < 0)
np.mean(fires_on_map.lon_idx < 0)

In [ ]:
fires_on_map.describe()

In [ ]:
fires_on_map.head()

In [ ]:
fires_on_map.info()

In [ ]:
fires_on_map.to_csv(f'{REGION}_firms.csv', index=False)

In [ ]:
fires_on_map.fire_cnt.value_counts()

In [ ]:
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'

# Check fire detections

In [ ]:
CHECK = True
if CHECK:
    M = np.load(f"maps/{REGION}-2020-08.npz")
    n_lats, n_lons = FIRECCI_SHAPE[SOUTH_AMERICA]

    l1 = M["l1"]
    X = cv2.resize(M["l1"], (n_lons, n_lats))
    l1.shape
    X.shape
    lats.shape, lons.shape

    fire_map = np.zeros(X.shape)
    fire_pixels = fires_on_map.groupby(['lat_idx', 'lon_idx']).fire_cnt.sum().reset_index()
    for i, j, v in tqdm(fire_pixels.values):
        fire_map[i, j] = v

    fire_map.mean()
    fire_map[fire_map > 1].mean()

In [ ]:
if CHECK:
    fig = plt.figure(figsize=(20, 20))
    plt.imshow(fire_map, cmap = plt.cm.inferno, vmin=0, vmax=2)
    plt.colorbar()
    plt.show();

In [ ]:
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'